In [11]:
### REQUIREMENTS
import pandas as pd
import sys
import json

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

import sddk

In [2]:
### configure session and groupurl
s = sddk.cloudSession("sciencedata.dk", "SDAM_root", "648597@au.dk")

connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [8]:
# to access our gsheet with data overview & metadata, you need a Google Service Account json file
# (see google-auth python package documentation to start with)
# I have mine located in my personal space on sciencedata.dk, so I read it from there:

# (1) read the file and parse its content
file_data = s.read_file("https://sciencedata.dk/files/ServiceAccountsKey.json", "dict")
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)
# (5) establish connection with spreadsheets specified by their url
EDH_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/164MLxVcCZg95Bzf9fVyD1-iCA5V97eM3KAFllyhTvt4/edit?usp=sharing")

# Merge the inscriptions with geographies

In [21]:
### load the API data either from sciencedata or locally:
### inscriptions_data_df = pd.DataFrame(s.get("https://sciencedata.dk/files/personal_folder/EDH_data/EDH_inscriptions_raw.json").json())
#inscriptions_data_df = s.read_file("SDAM_data/EDH/EDH_onebyone_2022-04-28.json")
inscriptions_data_df = pd.read_json("../data/large_data/EDH_onebyone.json")
inscriptions_data_df.head(5)

,commentary,country,depth,diplomatic_text,findspot_ancient,findspot_modern,height,id,language,last_update,...,transcription,trismegistos_uri,type_of_inscription,type_of_monument,width,work_status,findspot,year_of_find,present_location,religion
0,(C): 2. Hälfte 1. - Anfang 2. Jh. - AE; Ende 1...,Italy,2 cm,D M / NONIAE P F OPTATAE / ET C IVLIO ARTEMONI...,"Cumae, bei","Cuma, bei",33 cm,HD000001,L,2014-04-07,...,D(is) M(anibus) / Noniae P(ubli) f(iliae) Opta...,https://www.trismegistos.org/text/251193,epitaph,tabula,34 cm,provisional,None,NaN,None,None
1,AE 1983: Breite: 35 cm.,Italy,None,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,Roma,Roma,28 cm,HD000002,L,2014-04-07,...,C(aius) Sextius Paris / qui vixit / annis LXX,https://www.trismegistos.org/text/265631,epitaph,tabula,85 cm,no image,"Via Nomentana, S. Alessandro, Kirche",1937.0,None,None
2,(B): [S]isenna ist falscher Kasus; folgende Er...,Spain,(12) cm,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,None,Tomares,(37) cm,HD000003,L,2006-08-31,...,[P(ublio) M]ummio [P(ubli) f(ilio)] / [Gal(eri...,https://www.trismegistos.org/text/220675,honorific inscription,statue base,(34) cm,provisional,None,-1975.0,"Sevilla, Privatbesitz",None
3,Material: lokaler grauer Kalkstein. (B): Stylo...,Spain,18 cm,[ ]AVS[ ]LLA / M PORCI NIGRI SER / DOMINAE VEN...,Ipolcobulcula,Carcabuey,(39) cm,HD000004,L,2015-03-27,...,[---?]AV(?)S(?)[---]L(?)L(?)A / M(arci) Porci ...,https://www.trismegistos.org/text/222102,votive inscription,altar,27 cm,checked with photo,None,-1979.0,"Carcabuey, Grupo Escolar",names of pagan deities
4,(B): Z. 3: C(ai) l(ibertae) Tyches.,Italy,None,[ ] L SVCCESSVS / [ ] L L IRENAEVS / [ ] C L T...,Roma,Roma,None,HD000005,L,2010-01-04,...,[---] l(ibertus) Successus / [---] L(uci) l(ib...,https://www.trismegistos.org/text/265629,epitaph,stele,None,no image,Via Cupa (ehem. Vigna Nardi),NaN,None,None


In [44]:
inscriptions_data_df[inscriptions_data_df["trismegistos_uri"].isna()]

,commentary,country,depth,diplomatic_text,findspot_ancient,findspot_modern,height,id,language,last_update,...,transcription,trismegistos_uri,type_of_inscription,type_of_monument,width,work_status,findspot,year_of_find,present_location,religion
2874,Tafel wiederverwendet und auf der Rückseite mi...,Italy,6 cm,]IO M F / [ ]RIONI / [ ]ATVRNIN / [ ]RAE / [ /...,Vada Sabatia,Vado Ligure,(25) cm,HD002875,L,2012-05-07,...,------]io M(arci) f(ilio) / [---]rioni / [--- ...,None,epitaph,tabula,(21) cm,checked with photo,S. Pietro in Carpignano,1979.0,Soprintendenza archeologica della Liguria,None
4064,Vermutlich stammt die Inschrift vom Forum von ...,Italy,None,CAES AVG SAC / C SALLVSTIVS EPAGATHIC / OB HON...,Sinuessa,Cellole,None,HD004065,L,2011-09-23,...,Caes(ari) Aug(usto) sac(rum) / C(aius) Sallust...,None,votive inscription,None,None,provisional,Fondo Morrone,NaN,None,"cult functions, pagan"
8177,None,Spain,None,[ ] M S / [ ] PAVLINAE / [ ] LIBERTE AN / [ ] ...,"Toletum, bei",Polán,None,HD008178,L,1997-08-17,...,[D(is)] M(anibus) s(acrum) / [Pompeiae] Paulin...,None,None,None,None,provisional,None,NaN,None,None
8760,None,Italy,None,SVCESSI / / MIRAE PROSAPIAE ADQVE / NIMIAE INT...,Puteoli,Pozzuoli,None,HD008761,L,2009-10-22,...,Suc(c)essi // mirae prosapiae adque(!) / nimia...,None,None,None,None,provisional,None,NaN,None,None
9419,None,Italy,None,RVFO BONAE MEMO / RIAE QVI VIXIT / ANNIS SEX D...,None,Rossano,None,HD009420,L,1997-08-04,...,Rufo bonae memo/riae qui vixit / annis sex die...,None,None,None,None,provisional,None,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82819,None,Hungary,None,ITO PATER HRACVRA M[ ] / QCVRIS CVLENI EA NOMI...,Aquincum,Budapest,None,HD082820,L,2021-05-17,...,&lt;Dis=ITO&gt; Pater &lt;Ae=H&gt;racura M[er]...,None,defixio,tessera,None,provisional,"Óbuda, {Záhony utca 7}, Nekropole, Ausgrabung",2000.0,None,None
82820,None,Hungary,None,I O M / [ ]VNONI / [,Aquincum,Budapest,(52) cm,HD082821,L,2021-06-30,...,I(ovi) O(ptimo) M(aximo) / [I]unoni / [------,None,votive inscription,altar,66 cm,checked with photo,None,NaN,"Budapest, Aquincumi Múz.",names of pagan deities
82822,Inschrift eingepunzt.,Germany,None,| ALBANI,Colonia Ulpia Traiana,Xanten,None,HD082823,L,2021-09-24,...,|(centuria) Albani,None,None,instrumentum militare,None,checked with photo,None,NaN,"Xanten, LVR-Archäolog. Park Xanten / Römer-Mus.",None
82823,Scherbe eines Terra-Sigillata-Tellers (Typ Dra...,Germany,None,BRIGI,Grinario (Civitas Sumelocennensis),Köngen,None,HD082824,L,2021-11-05,...,Brigi,None,owner/artist inscription,instrumentum domesticum,None,checked with photo,None,NaN,None,None


In [24]:
EDH_xml_data = pd.read_json("../data/large_data/edh_xml_data.json")
EDH_xml_data.head()

,idno_uri,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,people
0,HD000001,NaN,"[{'type': 'ancient', 'text': 'Cumae, bei', 're...",\n Dis Manibus Noniae Publi filiae Optatae et ...,71 AD - 130 AD,NaN,NaN,257,tabula,NaN,"Marmor, ge\xc3\xa4dert / farbig",1000,92.0,epitaph,"[{'persname': {'name': [{'@type': 'nomen', '#t..."
1,HD000002,NaN,"[{'type': 'ancient', 'text': 'Roma', 'ref': 'h...",\n Caius Sextius Paris qui vixit annis LXX\n,51 AD - 200 AD,NaN,NaN,257,tabula,48.0,Marmor,1000,92.0,epitaph,"{'persname': {'name': [{'@type': 'praenomen', ..."
2,HD000003,NaN,"[{'type': 'ancient', 'text': '', 'ref': '#'}, ...",\n Publio Mummio Publi filio Galeria Sisennae ...,131 AD - 170 AD,NaN,NaN,57,statue base,48.0,Marmor,1000,69.0,honorific inscription,"{'persname': {'name': [{'@type': 'praenomen', ..."
3,HD000004,NaN,"[{'type': 'ancient', 'text': 'Ipolcobulcula', ...",\n AVSLLA Marci Porci Nigri serva dominae Vene...,151 AD - 200 AD,NaN,NaN,29,altar,60.0,Kalkstein,1000,372.0,votive inscription,"[{'persname': {'name': {'@type': 'cognomen', '..."
4,HD000005,NaN,"[{'type': 'ancient', 'text': 'Roma', 'ref': 'h...",\n libertus Successus Luci libertus Irenaeus C...,1 AD - 200 AD,NaN,NaN,250,stele,NaN,None,1000,92.0,epitaph,"[{'persname': {'name': [{'@type': 'praenomen',..."


In [54]:
def extract_pleiades_id(placenames_refs):
    try:
        return [refdict["ref"].rpartition("/")[2] for refdict in placenames_refs if "pleiades" in refdict["ref"]][0]
    except:
        return None

EDH_xml_data["pleiades_id"] = EDH_xml_data["placenames_refs"].apply(extract_pleiades_id)
EDH_xml_data.head(5)

,idno_uri,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,people,pleiades_id
0,HD000001,NaN,"[{'type': 'ancient', 'text': 'Cumae, bei', 're...",\n Dis Manibus Noniae Publi filiae Optatae et ...,71 AD - 130 AD,NaN,NaN,257,tabula,NaN,"Marmor, ge\xc3\xa4dert / farbig",1000,92.0,epitaph,"[{'persname': {'name': [{'@type': 'nomen', '#t...",432808
1,HD000002,NaN,"[{'type': 'ancient', 'text': 'Roma', 'ref': 'h...",\n Caius Sextius Paris qui vixit annis LXX\n,51 AD - 200 AD,NaN,NaN,257,tabula,48.0,Marmor,1000,92.0,epitaph,"{'persname': {'name': [{'@type': 'praenomen', ...",423025
2,HD000003,NaN,"[{'type': 'ancient', 'text': '', 'ref': '#'}, ...",\n Publio Mummio Publi filio Galeria Sisennae ...,131 AD - 170 AD,NaN,NaN,57,statue base,48.0,Marmor,1000,69.0,honorific inscription,"{'persname': {'name': [{'@type': 'praenomen', ...",None
3,HD000004,NaN,"[{'type': 'ancient', 'text': 'Ipolcobulcula', ...",\n AVSLLA Marci Porci Nigri serva dominae Vene...,151 AD - 200 AD,NaN,NaN,29,altar,60.0,Kalkstein,1000,372.0,votive inscription,"[{'persname': {'name': {'@type': 'cognomen', '...",None
4,HD000005,NaN,"[{'type': 'ancient', 'text': 'Roma', 'ref': 'h...",\n libertus Successus Luci libertus Irenaeus C...,1 AD - 200 AD,NaN,NaN,250,stele,NaN,None,1000,92.0,epitaph,"[{'persname': {'name': [{'@type': 'praenomen',...",423025


In [22]:
# load geographies
#EDH_geo_dict = s.read_file("SDAM_data/EDH/EDH_geo_dict_2022-04-28.json", "dict")
EDH_geo_dict = json.load(open("../data/large_data/geo_uri_dict.json", "r"))

In [23]:
EDH_geo_dict["432808"]

{'type': 'Feature',
 'properties': {'uri': 'https://edh-www.adw.uni-heidelberg.de/edh/geographie/12346',
  'pleiades_uri': 'https://pleiades.stoa.org/places/432808',
  'findspot_(village/street)': 'Via Campana antica',
  'modern_findspot': 'Cuma',
  'trismegistos_geo_uri': 'https://www.trismegistos.org/place/14437',
  'ancient_findspot': 'Cumae'},
 'geometry': {'coordinates': [14.0550756, 40.8471577], 'type': 'Point'}}

In [57]:
%%time

### use "edh_geography_uri" column to
### put information from geo_data_df "coordinates" column
### and return it as a new "coordinates" column in the inscriptions_data_df 

def get_coordinates(uri_value, latorlong):
  try:
    feature = EDH_geo_dict[uri_value.rpartition("/")[2]]
    if latorlong=="latitude":
        return feature["geometry"]["coordinates"][1]
    else:
        return feature["geometry"]["coordinates"][0]
  except:
    return None

EDH_xml_data["longitude"]  =  EDH_xml_data.apply(lambda row: get_coordinates(row["pleiades_id"], "longitude"), axis=1)

EDH_xml_data["latitude"]  =  EDH_xml_data.apply(lambda row: get_coordinates(row["pleiades_id"], "latitude"), axis=1)

CPU times: user 677 ms, sys: 98.2 ms, total: 776 ms
Wall time: 799 ms


In [58]:
EDH_xml_data.head(5)

,idno_uri,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,people,pleiades_id,longitude,latitude
0,HD000001,NaN,"[{'type': 'ancient', 'text': 'Cumae, bei', 're...",\n Dis Manibus Noniae Publi filiae Optatae et ...,71 AD - 130 AD,NaN,NaN,257,tabula,NaN,"Marmor, ge\xc3\xa4dert / farbig",1000,92.0,epitaph,"[{'persname': {'name': [{'@type': 'nomen', '#t...",432808,14.055076,40.847158
1,HD000002,NaN,"[{'type': 'ancient', 'text': 'Roma', 'ref': 'h...",\n Caius Sextius Paris qui vixit annis LXX\n,51 AD - 200 AD,NaN,NaN,257,tabula,48.0,Marmor,1000,92.0,epitaph,"{'persname': {'name': [{'@type': 'praenomen', ...",423025,12.486168,41.891737
2,HD000003,NaN,"[{'type': 'ancient', 'text': '', 'ref': '#'}, ...",\n Publio Mummio Publi filio Galeria Sisennae ...,131 AD - 170 AD,NaN,NaN,57,statue base,48.0,Marmor,1000,69.0,honorific inscription,"{'persname': {'name': [{'@type': 'praenomen', ...",None,NaN,NaN
3,HD000004,NaN,"[{'type': 'ancient', 'text': 'Ipolcobulcula', ...",\n AVSLLA Marci Porci Nigri serva dominae Vene...,151 AD - 200 AD,NaN,NaN,29,altar,60.0,Kalkstein,1000,372.0,votive inscription,"[{'persname': {'name': {'@type': 'cognomen', '...",None,NaN,NaN
4,HD000005,NaN,"[{'type': 'ancient', 'text': 'Roma', 'ref': 'h...",\n libertus Successus Luci libertus Irenaeus C...,1 AD - 200 AD,NaN,NaN,250,stele,NaN,None,1000,92.0,epitaph,"[{'persname': {'name': [{'@type': 'praenomen',...",423025,12.486168,41.891737


In [59]:
EDH_xml_data[EDH_xml_data["latitude"].notnull()]

,idno_uri,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,people,pleiades_id,longitude,latitude
0,HD000001,NaN,"[{'type': 'ancient', 'text': 'Cumae, bei', 're...",\n Dis Manibus Noniae Publi filiae Optatae et ...,71 AD - 130 AD,NaN,NaN,257,tabula,NaN,"Marmor, ge\xc3\xa4dert / farbig",1000,92.0,epitaph,"[{'persname': {'name': [{'@type': 'nomen', '#t...",432808,14.055076,40.847158
1,HD000002,NaN,"[{'type': 'ancient', 'text': 'Roma', 'ref': 'h...",\n Caius Sextius Paris qui vixit annis LXX\n,51 AD - 200 AD,NaN,NaN,257,tabula,48.0,Marmor,1000,92.0,epitaph,"{'persname': {'name': [{'@type': 'praenomen', ...",423025,12.486168,41.891737
4,HD000005,NaN,"[{'type': 'ancient', 'text': 'Roma', 'ref': 'h...",\n libertus Successus Luci libertus Irenaeus C...,1 AD - 200 AD,NaN,NaN,250,stele,NaN,None,1000,92.0,epitaph,"[{'persname': {'name': [{'@type': 'praenomen',...",423025,12.486168,41.891737
5,HD000006,NaN,"[{'type': 'ancient', 'text': 'Sabora, bei', 'r...",\n Dis Manibus sacrum Memmia Auctina annorum L...,71 AD - 150 AD,NaN,NaN,250,stele,60.0,Kalkstein,1000,92.0,epitaph,"{'persname': {'name': [{'@type': 'nomen', '#te...",256408,-5.024355,36.934635
6,HD000007,NaN,"[{'type': 'ancient', 'text': 'Roma', 'ref': 'h...",\n Clodia Marci filia\n,100 BC - 51 BC,NaN,NaN,257,tabula,71.0,Travertin,1000,92.0,epitaph,"{'persname': {'name': {'@type': 'nomen', '#tex...",423025,12.486168,41.891737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81873,HD082820,NaN,"[{'type': 'ancient', 'text': 'Aquincum', 'ref'...",\n &lt;Dis=ITO&gt; Pater &lt;Ae=H&gt;racura Me...,201 AD - 400 AD,NaN,NaN,276,tessera,108.0,Blei,1000,76.0,defixio,None,197130,18.661809,46.722776
81874,HD082821,NaN,"[{'type': 'ancient', 'text': 'Aquincum', 'ref'...",\n Iovi Optimo Maximo Iunoni \n,171 AD - 300 AD,NaN,NaN,29,altar,NaN,Gesteine,1000,372.0,votive inscription,None,197130,18.661809,46.722776
81875,HD082822,NaN,"[{'type': 'ancient', 'text': 'Colonia Claudia ...",\n M a&lt;m=S&gt;ator &lt;f=E&gt;eli&lt;x=C&gt...,1 AD - 300 AD,NaN,NaN,140,instrumentum domesticum,131.0,Ton,1000,73.0,acclamation,"{'persname': {'name': {'@type': 'cognomen', '#...",108751,6.868018,51.065052
81876,HD082823,NaN,"[{'type': 'ancient', 'text': 'Colonia Ulpia Tr...",\n centuria Albani\n,31 AD - 100 AD,NaN,NaN,141,instrumentum militare,109.0,Bronze,1000,143.0,unknown,"{'persname': {'name': {'@type': 'cognomen', '#...",108896,6.453261,51.662294


In [61]:
EDH_xml_data.rename(columns={"idno_uri":"id"}, inplace=True)

In [62]:
EDH_xml_data.head(5)

,id,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,people,pleiades_id,longitude,latitude
0,HD000001,NaN,"[{'type': 'ancient', 'text': 'Cumae, bei', 're...",\n Dis Manibus Noniae Publi filiae Optatae et ...,71 AD - 130 AD,NaN,NaN,257,tabula,NaN,"Marmor, ge\xc3\xa4dert / farbig",1000,92.0,epitaph,"[{'persname': {'name': [{'@type': 'nomen', '#t...",432808,14.055076,40.847158
1,HD000002,NaN,"[{'type': 'ancient', 'text': 'Roma', 'ref': 'h...",\n Caius Sextius Paris qui vixit annis LXX\n,51 AD - 200 AD,NaN,NaN,257,tabula,48.0,Marmor,1000,92.0,epitaph,"{'persname': {'name': [{'@type': 'praenomen', ...",423025,12.486168,41.891737
2,HD000003,NaN,"[{'type': 'ancient', 'text': '', 'ref': '#'}, ...",\n Publio Mummio Publi filio Galeria Sisennae ...,131 AD - 170 AD,NaN,NaN,57,statue base,48.0,Marmor,1000,69.0,honorific inscription,"{'persname': {'name': [{'@type': 'praenomen', ...",None,NaN,NaN
3,HD000004,NaN,"[{'type': 'ancient', 'text': 'Ipolcobulcula', ...",\n AVSLLA Marci Porci Nigri serva dominae Vene...,151 AD - 200 AD,NaN,NaN,29,altar,60.0,Kalkstein,1000,372.0,votive inscription,"[{'persname': {'name': {'@type': 'cognomen', '...",None,NaN,NaN
4,HD000005,NaN,"[{'type': 'ancient', 'text': 'Roma', 'ref': 'h...",\n libertus Successus Luci libertus Irenaeus C...,1 AD - 200 AD,NaN,NaN,250,stele,NaN,None,1000,92.0,epitaph,"[{'persname': {'name': [{'@type': 'praenomen',...",423025,12.486168,41.891737


In [64]:
inscriptions_rich = inscriptions_data_df.merge(EDH_xml_data, on="id", how="left")
inscriptions_rich.head(5)

,commentary,country,depth,diplomatic_text,findspot_ancient,findspot_modern,height,id,language,last_update,...,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,people,pleiades_id,longitude,latitude
0,(C): 2. Hälfte 1. - Anfang 2. Jh. - AE; Ende 1...,Italy,2 cm,D M / NONIAE P F OPTATAE / ET C IVLIO ARTEMONI...,"Cumae, bei","Cuma, bei",33 cm,HD000001,L,2014-04-07,...,tabula,NaN,"Marmor, ge\xc3\xa4dert / farbig",1000.0,92.0,epitaph,"[{'persname': {'name': [{'@type': 'nomen', '#t...",432808,14.055076,40.847158
1,AE 1983: Breite: 35 cm.,Italy,None,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,Roma,Roma,28 cm,HD000002,L,2014-04-07,...,tabula,48.0,Marmor,1000.0,92.0,epitaph,"{'persname': {'name': [{'@type': 'praenomen', ...",423025,12.486168,41.891737
2,(B): [S]isenna ist falscher Kasus; folgende Er...,Spain,(12) cm,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,None,Tomares,(37) cm,HD000003,L,2006-08-31,...,statue base,48.0,Marmor,1000.0,69.0,honorific inscription,"{'persname': {'name': [{'@type': 'praenomen', ...",None,NaN,NaN
3,Material: lokaler grauer Kalkstein. (B): Stylo...,Spain,18 cm,[ ]AVS[ ]LLA / M PORCI NIGRI SER / DOMINAE VEN...,Ipolcobulcula,Carcabuey,(39) cm,HD000004,L,2015-03-27,...,altar,60.0,Kalkstein,1000.0,372.0,votive inscription,"[{'persname': {'name': {'@type': 'cognomen', '...",None,NaN,NaN
4,(B): Z. 3: C(ai) l(ibertae) Tyches.,Italy,None,[ ] L SVCCESSVS / [ ] L L IRENAEVS / [ ] C L T...,Roma,Roma,None,HD000005,L,2010-01-04,...,stele,NaN,None,1000.0,92.0,epitaph,"[{'persname': {'name': [{'@type': 'praenomen',...",423025,12.486168,41.891737


In [65]:
inscriptions_rich.columns

Index(['commentary', 'country', 'depth', 'diplomatic_text', 'findspot_ancient',
       'findspot_modern', 'height', 'id', 'language', 'last_update',
       'letter_size', 'literature', 'material', 'modern_region', 'not_after',
       'not_before', 'responsible_individual', 'transcription',
       'trismegistos_uri', 'type_of_inscription', 'type_of_monument', 'width',
       'work_status', 'findspot', 'year_of_find', 'present_location',
       'religion', 'idno_tm', 'placenames_refs', 'text_edition',
       'origdate_text', 'layout_execution', 'layout_execution_text',
       'support_objecttype', 'support_objecttype_text', 'support_material',
       'support_material_text', 'support_decoration', 'keywords_term',
       'keywords_term_text', 'people', 'pleiades_id', 'longitude', 'latitude'],
      dtype='object')

In [66]:
# last time we had (80270, 41)
inscriptions_rich.shape

(81883, 44)

In [67]:
EDH_merged_columns = pd.DataFrame(pd.DataFrame(inscriptions_rich).columns, columns=["columns"])
EDH_merged_columns

,columns
0,commentary
1,country
2,depth
3,diplomatic_text
4,findspot_ancient
5,findspot_modern
6,height
7,id
8,language
9,last_update


In [27]:
# uncomment to export to gsheets
set_with_dataframe(EDH_overview.add_worksheet("EDH_merged_columns", 1, 1), EDH_merged_columns)

# Upload the merged data to sciencedata.dk

In [25]:
%%time
### put your dataframe data into this folder

sddk.write_file("SDAM_data/EDH/public/EDH_merged_2020-09-22.json", inscriptions_rich, conf)

A file with the same name ("EDH_merged_2020-09-22.json") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/EDH/public/EDH_merged_2020-09-22.json"
CPU times: user 4.01 s, sys: 1.86 s, total: 5.88 s
Wall time: 1min 6s


# Reading the file back and exporting a feather file (UNDER DEVELOPMENT)

In [39]:
inscriptions_rich = sddk.read_file("SDAM_data/EDH/EDH_merged_2020-09-22.json", "df", conf)
inscriptions_rich.head(5)

,responsible_individual,type_of_inscription,letter_size,not_after,literature,work_status,height,diplomatic_text,people,depth,...,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text
0,Feraudi,epitaph,3.2-2 cm,0130,"AE 1983, 0192.; M. Annecchino, Puteoli 4/5, 19...",provisional,33 cm,D M / NONIAE P F OPTATAE / ET C IVLIO ARTEMONI...,"[{'name': 'Noniae P.f. Optatae', 'cognomen': '...",2.7 cm,...,None,None,None,None,None,None,None,None,None,None
1,Feraudi,epitaph,4 cm,0200,"AE 1983, 0080. (A); A. Ferrua, RAL 36, 1981, 1...",no image,28 cm,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,"[{'nomen': 'Sextius', 'praenomen': 'C.', 'pers...",None,...,51 AD – 200 AD,21,unbestimmt,257,Tafel,48,Marmor,1000,92,Grabinschrift
2,Feraudi,honorific inscription,4.5-3 cm,0170,"AE 1983, 0518. (B); J. González, ZPE 52, 1983,...",provisional,(37) cm,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,"[{'nomen': 'Mummius+', 'gender': 'male', 'prae...",(12) cm,...,131 AD – 170 AD,21,unbestimmt,57,Statuenbasis,48,Marmor,1000,69,Ehreninschrift
3,Gräf,votive inscription,2.5 cm,0200,"AE 1983, 0533. (B); A.U. Stylow, Gerión 1, 198...",checked with photo,(39) cm,[ ]AVS[ ]LLA / M PORCI NIGRI SER / DOMINAE VEN...,[{'name': '[---?]AV(?)S(?)[---]L(?)L(?)A M. Po...,18 cm,...,151 AD – 200 AD,21,unbestimmt,29,Altar,60,Kalkstein,1000,80,Weihinschrift
4,Feraudi,epitaph,None,0200,"AE 1983, 0078. (B); A. Ferrua, RAL 36, 1981, 1...",no image,None,[ ] L SVCCESSVS / [ ] L L IRENAEVS / [ ] C L T...,"[{'person_id': '1', 'name': '[---]l. Successus...",None,...,1 AD – 200 AD,21,unbestimmt,250,Stele,138,unbestimmt,1000,92,Grabinschrift


In [43]:
sys.getsizeof(inscriptions_rich)

297272815

In [44]:
for column in inscriptions_rich.columns: # to avoid problems with encoding, lets check that everything is utf-8
    try: 
        inscriptions_rich[column] = inscriptions_rich[column].str.encode("utf-8")
        inscriptions_rich[column] = inscriptions_rich[column].str.decode("utf-8")
    except: 
        inscriptions_rich[column] = inscriptions_rich[column]  
inscriptions_rich.head(5)

,responsible_individual,type_of_inscription,letter_size,not_after,literature,work_status,height,diplomatic_text,people,depth,...,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text
0,Feraudi,epitaph,3.2-2 cm,0130,"AE 1983, 0192.; M. Annecchino, Puteoli 4/5, 19...",provisional,33 cm,D M / NONIAE P F OPTATAE / ET C IVLIO ARTEMONI...,NaN,2.7 cm,...,None,None,None,None,None,None,None,None,None,None
1,Feraudi,epitaph,4 cm,0200,"AE 1983, 0080. (A); A. Ferrua, RAL 36, 1981, 1...",no image,28 cm,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,NaN,None,...,51 AD – 200 AD,21,unbestimmt,257,Tafel,48,Marmor,1000,92,Grabinschrift
2,Feraudi,honorific inscription,4.5-3 cm,0170,"AE 1983, 0518. (B); J. González, ZPE 52, 1983,...",provisional,(37) cm,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,NaN,(12) cm,...,131 AD – 170 AD,21,unbestimmt,57,Statuenbasis,48,Marmor,1000,69,Ehreninschrift
3,Gräf,votive inscription,2.5 cm,0200,"AE 1983, 0533. (B); A.U. Stylow, Gerión 1, 198...",checked with photo,(39) cm,[ ]AVS[ ]LLA / M PORCI NIGRI SER / DOMINAE VEN...,NaN,18 cm,...,151 AD – 200 AD,21,unbestimmt,29,Altar,60,Kalkstein,1000,80,Weihinschrift
4,Feraudi,epitaph,None,0200,"AE 1983, 0078. (B); A. Ferrua, RAL 36, 1981, 1...",no image,None,[ ] L SVCCESSVS / [ ] L L IRENAEVS / [ ] C L T...,NaN,None,...,1 AD – 200 AD,21,unbestimmt,250,Stele,138,unbestimmt,1000,92,Grabinschrift


In [45]:
sys.getsizeof(inscriptions_rich)

276460615

In [46]:
inscriptions_rich_sample = inscriptions_rich[:5000]

In [47]:
len(inscriptions_rich_sample)

5000

In [48]:
sddk.write_file("SDAM_data/EDH/public/EDH_utf_sample_2.json", inscriptions_rich_sample, conf) 

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/EDH/public/EDH_utf_sample_2.json"


In [12]:
from pyarrow import feather

In [20]:
pyarrow.feather.write_feather(inscriptions_rich.reset_index(), "test-utf8.feather", version=1, compression=None)

In [21]:
conf[0].put(conf[1] + "SDAM_data/EDH/test-utf8.feather", data=open("test-utf8.feather", "rb"))

<Response [201]>

In [17]:
EDH_feather = pd.read_feather("test-utf8.feather")
for column in EDH_feather.columns:
    try:
        EDH_feather[column] = EDH_feather[column].str.decode("utf-8")
    except:
        EDH_feather[column] = EDH_feather[column]
EDH_feather.head()

,index,responsible_individual,type_of_inscription,letter_size,not_after,literature,work_status,height,diplomatic_text,people,...,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,support_decoration_text,keywords_term,keywords_term_text
0,NaN,Feraudi,epitaph,3.2-2 cm,0130,"AE 1983, 0192.; M. Annecchino, Puteoli 4/5, 19...",provisional,33 cm,D M / NONIAE P F OPTATAE / ET C IVLIO ARTEMONI...,NaN,...,NaN,None,None,None,None,None,NaN,None,None,None
1,NaN,Feraudi,epitaph,4 cm,0200,"AE 1983, 0080. (A); A. Ferrua, RAL 36, 1981, 1...",no image,28 cm,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,NaN,...,NaN,unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/257,Tafel,http://www.eagle-network.eu/voc/material/lod/48,Marmor,NaN,nein,http://www.eagle-network.eu/voc/typeins/lod/92,Grabinschrift
2,NaN,Feraudi,honorific inscription,4.5-3 cm,0170,"AE 1983, 0518. (B); J. González, ZPE 52, 1983,...",provisional,(37) cm,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,NaN,...,NaN,unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/57,Statuenbasis,http://www.eagle-network.eu/voc/material/lod/48,Marmor,NaN,nein,http://www.eagle-network.eu/voc/typeins/lod/69,Ehreninschrift
3,NaN,Gräf,votive inscription,2.5 cm,0200,"AE 1983, 0533. (B); A.U. Stylow, Gerión 1, 198...",checked with photo,(39) cm,[ ]AVS[ ]LLA / M PORCI NIGRI SER / DOMINAE VEN...,NaN,...,NaN,unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/29,Altar,http://www.eagle-network.eu/voc/material/lod/60,Kalkstein,NaN,nein,http://www.eagle-network.eu/voc/typeins/lod/80,Weihinschrift
4,NaN,Feraudi,epitaph,None,0200,"AE 1983, 0078. (B); A. Ferrua, RAL 36, 1981, 1...",no image,None,[ ] L SVCCESSVS / [ ] L L IRENAEVS / [ ] C L T...,NaN,...,NaN,unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/250,Stele,http://www.eagle-network.eu/voc/material/lod/138,unbestimmt,NaN,nein,http://www.eagle-network.eu/voc/typeins/lod/92,Grabinschrift


In [15]:
sddk.write_file("SDAM_data/EDH/EDH_merged_2020-09-18.feather", inscriptions_rich.reset_index(), conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/EDH/EDH_merged_2020-09-18.feather"


In [16]:
EDH_feather = sddk.read_file("SDAM_data/EDH/EDH_merged_2020-09-18.feather", "df", conf)

In [17]:
EDH_feather.head(5)

,index,responsible_individual,type_of_inscription,letter_size,not_after,literature,work_status,height,diplomatic_text,people,...,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,support_decoration_text,keywords_term,keywords_term_text
0,0,Feraudi,epitaph,3.2-2 cm,0130,"AE 1983, 0192.; M. Annecchino, Puteoli 4/5, 19...",provisional,33 cm,D M / NONIAE P F OPTATAE / ET C IVLIO ARTEMONI...,NaN,...,NaN,None,None,None,None,None,NaN,None,None,None
1,1,Feraudi,epitaph,4 cm,0200,"AE 1983, 0080. (A); A. Ferrua, RAL 36, 1981, 1...",no image,28 cm,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,NaN,...,NaN,unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/257,Tafel,http://www.eagle-network.eu/voc/material/lod/48,Marmor,NaN,nein,http://www.eagle-network.eu/voc/typeins/lod/92,Grabinschrift
2,2,Feraudi,honorific inscription,4.5-3 cm,0170,"AE 1983, 0518. (B); J. González, ZPE 52, 1983,...",provisional,(37) cm,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,NaN,...,NaN,unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/57,Statuenbasis,http://www.eagle-network.eu/voc/material/lod/48,Marmor,NaN,nein,http://www.eagle-network.eu/voc/typeins/lod/69,Ehreninschrift
3,3,Gräf,votive inscription,2.5 cm,0200,"AE 1983, 0533. (B); A.U. Stylow, Gerión 1, 198...",checked with photo,(39) cm,[ ]AVS[ ]LLA / M PORCI NIGRI SER / DOMINAE VEN...,NaN,...,NaN,unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/29,Altar,http://www.eagle-network.eu/voc/material/lod/60,Kalkstein,NaN,nein,http://www.eagle-network.eu/voc/typeins/lod/80,Weihinschrift
4,4,Feraudi,epitaph,None,0200,"AE 1983, 0078. (B); A. Ferrua, RAL 36, 1981, 1...",no image,None,[ ] L SVCCESSVS / [ ] L L IRENAEVS / [ ] C L T...,NaN,...,NaN,unbestimmt,http://www.eagle-network.eu/voc/objtyp/lod/250,Stele,http://www.eagle-network.eu/voc/material/lod/138,unbestimmt,NaN,nein,http://www.eagle-network.eu/voc/typeins/lod/92,Grabinschrift
